Steps to update raw hospital data:

1. Follow the README of https://dagshub.com/onefact/data.payless.health 

```
git clone https://dagshub.com/onefact/data.payless.health.git
cd data.payless.health
dvc pull
```

2. Run this script to download the files sequentially.
3. In the DVC repository, execute:

```
dvc add hospital_price_transparency_data/raw
dvc push
git commit -m 'update d
ata'
```

In [2]:
import pandas as pd
import wget 
import os
from urllib.parse import urlparse
from pathlib import Path
price_urls_dir = Path("/Users/me/tmp/data.payless.health/hospital_price_transparency_data/price_urls")
raw_data_dir = Path("/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw")

In [3]:
df = pd.read_csv(price_urls_dir / "220106-dolthub.com-onefact-paylesshealth-hospitals.csv")

In [3]:
df.head()

,ccn,name,state,city,homepage,cdm_indirect_url,cdm_url
0,154063,"NEUROPSYCHIATRIC HOSPITAL OF INDIANAPOLIS, LLC",IN,INDIANAPOLIS,https://www.neuropsychiatrichospitals.net/,https://www.neuropsychiatrichospitals.net/pric...,https://www.neuropsychiatrichospitals.net/wp-c...
1,154064,ASSURANCE HEALTH PSYCHIATRIC HOSPITAL,IN,INDIANAPOLIS,http://assurancehealthsystem.com/,NaN,NaN
2,154065,NEURO BEHAVIORAL HOSPITAL,IN,CROWN POINT,https://www.neuropsychiatrichospitals.net/,https://www.neuropsychiatrichospitals.net/pric...,https://www.neuropsychiatrichospitals.net/wp-c...
3,154066,BRIGHTWELL BEHAVIORAL HEALTH,IN,CLARKSVILLE,https://brightwellbehavioral.com/,NaN,NaN
4,154067,HENDRICKS BEHAVIORAL HOSPITAL,IN,PLAINFIELD,https://www.hendricksbehavioralhospital.com/,https://www.hendricksbehavioralhospital.com/,https://s3.amazonaws.com/ycubaa-production-mar...


In [5]:

error_rows = []
for _, df_row in df.iterrows():
    if type(df_row["cdm_url"]) == str:
        filename = os.path.basename(urlparse(df_row["cdm_url"]).path)
        out_path = raw_data_dir / f"{df_row['ccn']}_{filename}"
        if out_path.exists():
            pass
        else:
            try:
                print(out_path)
                wget.download(df_row["cdm_url"], out=str(out_path))
            except:
                error_rows.append(df_row)

/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw/160001_Marshalltown.xlsx
/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw/160013_Muscatine.xlsx
/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw/160016_Fort%20Dodge.xlsx
/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw/160029_MHIC%20Standard%20CDM%20Updated%2010.31.2022.xlsx
/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw/160030_421347891_mary-greeley-medical-center_standardcharges.xlsx
/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw/160033_421418847_genesis-health-system-davenport_standardcharges.csv
/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw/160045_Cedar%20Rapids.xlsx
/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw/160079_420698295_MERCY-MEDICAL-CENTER---CEDAR-RAPIDS_STANDARD-CHARGES.xlsx
/Users/me/tmp/data.payless.health/hospital_price_transparency_data

In [14]:
import os
import asyncio
import aiohttp  # pip install aiohttp
import aiofiles  # pip install aiofiles


def get_filename(url: str, ccn: str) -> str:
    remote_filename = os.path.basename(urlparse(url).path)
    return f"{ccn}_{filename}"


def download_files(url_list: list, ccn_list: list, raw_data_dir: str):
    sema = asyncio.BoundedSemaphore(5)

    async def fetch_file(url, ccn):
        fname = get_filename(url, ccn)
        if os.path.exists(os.path.join(raw_data_dir, fname)):
            print("exists")
            return
        async with sema, aiohttp.ClientSession() as session:
            async with session.get(url) as resp:
                assert resp.status == 200
                data = await resp.read()

        async with aiofiles.open(os.path.join(raw_data_dir, fname), "wb") as outfile:
            await outfile.write(data)

    loop = asyncio.get_event_loop()
    tasks = [loop.create_task(fetch_file(url, ccn)) for url, ccn in zip(url_list, ccn_list) if type(url) == str]
    loop.run_until_complete(asyncio.wait(tasks))
    loop.close()
    


In [15]:
await download_files(df["cdm_url"], df["ccn"], raw_data_dir)

Task exception was never retrieved
future: <Task finished name='Task-21546' coro=<download_files.<locals>.fetch_file() done, defined at /var/folders/2c/hmpw9qgd7fd3x0ms6548_f880000gn/T/ipykernel_13045/2878094845.py:15> exception=NameError("name 'filename' is not defined")>
Traceback (most recent call last):
  File "/var/folders/2c/hmpw9qgd7fd3x0ms6548_f880000gn/T/ipykernel_13045/2878094845.py", line 16, in fetch_file
    fname = get_filename(url, ccn)
  File "/var/folders/2c/hmpw9qgd7fd3x0ms6548_f880000gn/T/ipykernel_13045/2878094845.py", line 9, in get_filename
    return f"{ccn}_{filename}"
NameError: name 'filename' is not defined
/opt/homebrew/anaconda3/envs/payless.health/lib/python3.10/asyncio/base_events.py:438: RuntimeWarning: coroutine 'wait' was never awaited
  task = tasks.Task(coro, loop=self, name=name)
Task exception was never retrieved
future: <Task finished name='Task-21547' coro=<download_files.<locals>.fetch_file() done, defined at /var/folders/2c/hmpw9qgd7fd3x0ms6548